# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [36]:
# load data from database
engine = create_engine('sqlite:///MessagesClean.db')
df = pd.read_sql("SELECT * FROM messages_clean", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [37]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26108,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26109,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26110,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26111,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [38]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9}]', ' ', text.lower())
    tokens = word_tokenize(text)
    stop_words = stopwords.words('english')
    lemmatizer =  WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [40]:
tokenize(df['message'][0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [41]:
df['message'][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [43]:
pipeline = Pipeline([('vetc', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=43)

np.random.seed(20)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vetc',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000018C19794438>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [47]:
y_pred = pipeline.predict(X_train)
y_true = np.array(y_train)
target_names = list(Y.columns.values)

In [48]:
y_pred

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [49]:
y_true

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [50]:
y_pred.size, y_true.size

(705024, 705024)

In [51]:
target_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

Considering it is a multiclass-multioutput, I have to create a for loop to display the results in a DataFrame.

In [73]:
def display_results(y_test, y_pred, col_names):
    results = []
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
        precision = precision_score(y_test[:, i], y_pred[:, i], average='weighted')
        recall = recall_score(y_test[:, i], y_pred[:, i], average='weighted')
        f1 = f1_score(y_test[:, i], y_pred[:, i], average='weighted')
        results.append([accuracy, precision, recall, f1])
    
    results = np.array(results)
    results_df = pd.DataFrame(data=results, index=col_names, columns=['Accuracy', 'Precision', 'Recall', 'F1'])  
    return results_df

Below, I will evaluate the results of the train set:

In [74]:
y_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)
results_train = pd.DataFrame(display_results(np.array(y_train), y_pred, col_names))
results_train

,Accuracy,Precision,Recall,F1
related,0.998417,0.998418,0.998417,0.998416
request,0.999234,0.999234,0.999234,0.999234
offer,1.000000,1.000000,1.000000,1.000000
aid_related,0.999183,0.999183,0.999183,0.999183
medical_help,0.999745,0.999745,0.999745,0.999745
medical_products,0.999847,0.999847,0.999847,0.999847
search_and_rescue,0.999847,0.999847,0.999847,0.999847
security,0.999847,0.999847,0.999847,0.999847
military,0.999949,0.999949,0.999949,0.999949
child_alone,1.000000,1.000000,1.000000,1.000000


In [75]:
results_train.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.999769,0.999769,0.999769,0.999769
std,0.000322,0.000322,0.000322,0.000322
min,0.998417,0.998418,0.998417,0.998416
25%,0.999745,0.999745,0.999745,0.999745
50%,0.999898,0.999898,0.999898,0.999898
75%,0.999949,0.999949,0.999949,0.999949
max,1.000000,1.000000,1.000000,1.000000


And the results for the test set:

In [76]:
y_pred2 = pipeline.predict(X_test)
col_names = list(Y.columns.values)
results_test = pd.DataFrame(display_results(np.array(y_test), y_pred2, col_names))
results_test

C:\Users\ROLD003\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1
related,0.814214,0.801856,0.814214,0.795510
request,0.901363,0.895849,0.901363,0.891726
offer,0.995558,0.991136,0.995558,0.993342
aid_related,0.780824,0.779288,0.780824,0.779321
medical_help,0.923725,0.900738,0.923725,0.894884
medical_products,0.953745,0.942880,0.953745,0.934445
search_and_rescue,0.973197,0.971254,0.973197,0.961364
security,0.983612,0.968090,0.983612,0.975789
military,0.969521,0.966442,0.969521,0.956544
child_alone,1.000000,1.000000,1.000000,1.000000


In [77]:
results_test.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.948861,0.938138,0.948861,0.936996
std,0.050854,0.054553,0.050854,0.055490
min,0.780824,0.779288,0.780824,0.779321
25%,0.938620,0.925271,0.938620,0.923413
50%,0.961097,0.951606,0.961097,0.946948
75%,0.984339,0.977821,0.984339,0.977166
max,1.000000,1.000000,1.000000,1.000000


According to the Sklearn metrics, the ML model is working very well for the train and test sets. A poor model will show results close to zero and a good model will show results close to 1.

- Accuracy: In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.
- Precision: The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
- Recall: The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
- F1: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is: F1 = 2 * (precision * recall) / (precision + recall). In the multi-class and multi-label case, this is the average of the F1 score of each class with weighting depending on the average parameter.

Source: https://scikit-learn.org/stable/modules/model_evaluation.html

### 6. Improve your model
Use grid search to find better parameters. 

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [94]:
pipeline.get_params()

{'memory': None,
 'steps': [('vetc',
   CountVectorizer(tokenizer=<function tokenize at 0x0000018C19794438>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vetc': CountVectorizer(tokenizer=<function tokenize at 0x0000018C19794438>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vetc__analyzer': 'word',
 'vetc__binary': False,
 'vetc__decode_error': 'strict',
 'vetc__dtype': numpy.int64,
 'vetc__encoding': 'utf-8',
 'vetc__input': 'content',
 'vetc__lowercase': True,
 'vetc__max_df': 1.0,
 'vetc__max_features': None,
 'vetc__min_df': 1,
 'vetc__ngram_range': (1, 1),
 'vetc__preprocessor': None,
 'vetc__stop_words': None,
 'vetc__strip_accents': None,
 'vetc__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vetc__tokenizer': <function __main__.tokenize(text)>,
 'vetc__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [96]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vetc', 'tfidf', 'clf', 'vetc__analyzer', 'vetc__binary', 'vetc__decode_error', 'vetc__dtype', 'vetc__encoding', 'vetc__input', 'vetc__lowercase', 'vetc__max_df', 'vetc__max_features', 'vetc__min_df', 'vetc__ngram_range', 'vetc__preprocessor', 'vetc__stop_words', 'vetc__strip_accents', 'vetc__token_pattern', 'vetc__tokenizer', 'vetc__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [116]:
parameters = {'vetc__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25],
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
best_model = cv.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1, score=0.238, total=  35.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.1s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1, score=0.242, total=  35.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=5, score=0.243, total=  28.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1, score=0.231, total=  38.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1, score=0.245, total=  36.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vetc__min_df=1, score=0.235, total=  35.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, v

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1, score=0.224, total=  28.2s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1, score=0.243, total=  28.3s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=1, score=0.237, total=  28.5s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__min_df=5, score=0.234, total=  25.3s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vetc__mi

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5, score=0.252, total=  46.0s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5, score=0.268, total=  47.3s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5, score=0.261, total=  47.7s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=5, score=0.271, total=  49.6s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, 

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vetc__min_df=5, score=0.271, total= 2.1min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vetc__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vetc__min_df=5, score=0.267, total= 2.1min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=1, score=0.231, total= 2.3min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vetc__min_df=1, score=0.251, total= 2.2min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=Fa

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 99.3min finished


In [117]:
best_model.cv_results_

{'mean_fit_time': array([ 32.60170646,  26.34961991,  32.72150431,  25.01782575,
         72.55554509,  56.9293117 ,  71.09916787,  51.06011801,
         25.52886467,  22.70656428,  25.39513593,  21.75140676,
         50.92153258,  43.97360048,  52.5595438 ,  44.64342818,
         24.31956763,  22.46885924,  23.09789243,  21.74657688,
         46.77770867,  84.14018316, 124.64360127,  79.79816194]),
 'std_fit_time': array([ 0.55534489,  0.48406968,  0.91179008,  0.32550582,  1.45986943,
         0.91910323,  2.69542477,  2.16001744,  0.31791463,  0.40328252,
         0.20001215,  0.49149445,  0.21875963,  0.56918631,  1.45566545,
         2.85776845,  0.39999521,  0.70191508,  0.45028564,  1.5131044 ,
         0.31773661, 32.75022254,  1.80642002,  1.21208172]),
 'mean_score_time': array([ 3.16495738,  2.99904003,  3.25816197,  2.86846242,  5.20881176,
         4.5630424 ,  4.96962366,  4.14233751,  3.07980218,  3.10146375,
         3.0163538 ,  2.78882365,  4.61622181,  4.02089543,  4

In [118]:
np.max(best_model.cv_results_['mean_test_score'])

0.26388906635985776

In [119]:
best_model.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': False,
 'vetc__min_df': 5}

In [120]:
y_pred3 = best_model.predict(X_test)

col_names = list(Y.columns.values)
results_test3 = pd.DataFrame(display_results(np.array(y_test), y_pred3, col_names))
results_test3

C:\Users\ROLD003\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1
related,0.811457,0.798432,0.811457,0.797342
request,0.899832,0.893674,0.899832,0.890475
offer,0.995558,0.991136,0.995558,0.993342
aid_related,0.770562,0.772239,0.770562,0.771204
medical_help,0.921581,0.895582,0.921581,0.899044
medical_products,0.958493,0.954598,0.958493,0.944568
search_and_rescue,0.973043,0.966233,0.973043,0.962030
security,0.982999,0.970236,0.982999,0.975766
military,0.969521,0.962039,0.969521,0.957971
child_alone,1.000000,1.000000,1.000000,1.000000


In [121]:
results_test3.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.949805,0.939804,0.949805,0.939839
std,0.051902,0.055533,0.051902,0.055694
min,0.770562,0.772239,0.770562,0.771204
25%,0.944670,0.923308,0.944670,0.930578
50%,0.962935,0.955015,0.962935,0.952405
75%,0.984109,0.977484,0.984109,0.977468
max,1.000000,1.000000,1.000000,1.000000


This model is slightly better than the old model. However, the improvement is immaterial.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.